In [2]:
import pandas as pd

### Infos du csv

In [3]:

df = pd.read_csv('data/earthquakes.csv')

print('Shape:', df.shape)

pd.set_option('display.max_columns', None)

df.head()

Shape: (3272774, 22)


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1970-01-01T00:00:00.0Z,37.003502,-117.996834,0.0,0.00,mh,0.0,NaN,NaN,NaN,ci,ci37038459,2016-04-02T20:22:05.312Z,"29km NE of Independence, CA",sonic boom,NaN,NaN,NaN,0.0,reviewed,ci,ci
1,1970-01-01T00:00:00.0Z,35.642788,-120.933601,5.0,1.99,mh,2.0,NaN,NaN,NaN,ci,ci11092098,2016-01-29T01:43:14.870Z,"11km SSW of Lake Nacimiento, CA",earthquake,NaN,NaN,NaN,0.0,reviewed,ci,ci
2,1970-01-01T00:00:00.0Z,34.164520,-118.185036,0.0,0.00,mh,NaN,NaN,NaN,NaN,ci,ci15086796,2016-04-02T17:20:31.235Z,"4km S of La Canada Flintridge, CA",earthquake,NaN,NaN,NaN,0.0,reviewed,ci,ci
3,1970-01-01T00:00:00.0Z,33.836494,-116.781868,0.0,0.00,mh,NaN,NaN,NaN,NaN,ci,ci14891508,2016-04-02T14:10:48.389Z,"9km S of Cabazon, CA",sonic boom,NaN,NaN,NaN,0.0,reviewed,ci,ci
4,1970-01-01T00:00:00.0Z,33.208477,-115.476997,5.0,0.00,mh,NaN,NaN,NaN,NaN,ci,ci10925125,2016-04-02T04:32:22.103Z,"5km SE of Niland, CA",sonic boom,NaN,NaN,NaN,0.0,reviewed,ci,ci


### Types de variables avant conversion

In [ ]:
df = pd.read_csv('data/earthquakes.csv')
df.dtypes

### Conversion des types des variables et création du parquet


In [ ]:
df = pd.read_csv('data/earthquakes.csv')

# tout convertir
df = df.convert_dtypes()

# puis les dates
date_columns = ['time', 'updated']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], utc=True, errors='coerce')

df.to_parquet('data/STEP01_earthquakes.parquet')

In [ ]:
df = pd.read_parquet('data/STEP01_earthquakes.parquet')
df.dtypes

### Vérification conversion des dates

In [ ]:
df = pd.read_parquet('data/STEP01_earthquakes.parquet')

# Compter les valeurs vides dans 'time' et 'updated'
time_nulls = df['time'].isnull().sum()
updated_nulls = df['updated'].isnull().sum()

print(f"Nombre de valeurs vides dans 'time': {time_nulls}")
print(f"Nombre de valeurs vides dans 'updated': {updated_nulls}")

# Compter le nombre de lignes du fichier
total_rows = len(df)
print(f"Nombre total de lignes dans le fichier: {total_rows}")

# Compter le nombre de variables de type date (non-null) dans 'time' et 'updated'
time_date_count = df['time'].notnull().sum()
updated_date_count = df['updated'].notnull().sum()

print(f"Nombre de variables de type date dans 'time': {time_date_count}")
print(f"Nombre de variables de type date dans 'updated': {updated_date_count}")

### Doublons ?

In [ ]:
# Check for strictly identical rows
df = pd.read_parquet('data/STEP01_earthquakes.parquet')

num_duplicates = df.duplicated().sum()
print(f'Number of strictly identical rows: {num_duplicates}')

if num_duplicates > 0:
    duplicated_rows = df[df.duplicated(keep=False)]
    display(duplicated_rows.head(10)) 
else:
    print('No duplicate rows found.')

In [ ]:
# Check for duplicate 'id' values
df = pd.read_parquet('data/STEP01_earthquakes.parquet')
total_ids = len(df)
unique_ids = df['id'].nunique()
duplicate_ids_count = total_ids - unique_ids

print(f"Total 'id' entries: {total_ids}")
print(f"Unique 'id' entries: {unique_ids}")
print(f"Number of identical 'id's (duplicates): {duplicate_ids_count}")

In [ ]:
df = pd.read_parquet('data/STEP01_earthquakes.parquet')

before = len(df)

# Supprimer les doublons
df = df.drop_duplicates()

after = len(df)

# Calculer le nombre de doublons supprimés
removed = before - after

df.to_parquet('data/STEP02_earthquakes.parquet')

print(f"Fichier 'data/STEP02_earthquakes.parquet' créé sans doublons.")
print(f"Nombre de doublons supprimés : {removed}")


### Les valeurs 0 de nst et magNst passent en valeurs vides

In [ ]:
df = pd.read_parquet('data/STEP02_earthquakes.parquet')

# Remplacer les 0 par NaN dans nst et magNst
df['nst'] = df['nst'].replace(0, pd.NA)
df['magNst'] = df['magNst'].replace(0, pd.NA)

# Sauvegarder le parquet mis à jour
df.to_parquet('data/STEP03_earthquakes.parquet')

### Vérification valeurs faussement différentes dans "place" (ex : central East Pacific Rise, Central East Pacific Rise)

In [ ]:
df = pd.read_parquet('data/STEP03_earthquakes.parquet')

# Nombre de valeurs uniques dans 'place' original
unique_original = df['place'].nunique()
print(f"Nombre de valeurs uniques dans 'place' original : {unique_original}")

# Compter les occurrences de chaque place
place_counts = df['place'].value_counts()

# Places qui apparaissent plus d'une fois
duplicates = place_counts[place_counts > 1]

print(f"Nombre de places qui apparaissent plus d'une fois : {len(duplicates)}")
print(f"Total d'occurrences pour ces places : {duplicates.sum()}")

if len(duplicates) > 0:
    print("Exemples de places avec leurs occurrences :")
    print(duplicates.head(10))

In [ ]:
import unicodedata
import re

def clean_place(place):
    if pd.isna(place):
        return place
    # Remove accents
    place = unicodedata.normalize('NFD', place).encode('ascii', 'ignore').decode('ascii')
    # Convert to lowercase
    place = place.lower()
    # Remove punctuation
    place = re.sub(r'[^\w\s]', '', place)
    # Remove extra spaces
    place = ' '.join(place.split())
    return place

df = pd.read_parquet('data/STEP03_earthquakes.parquet')

# Apply cleaning to the 'place' column
df['place_cleaned'] = df['place'].apply(clean_place)

# Count unique cleaned places
unique_clean = df['place_cleaned'].nunique()
print(f"Nombre de valeurs uniques dans 'place' après nettoyage : {unique_clean}")

# Group by cleaned place and show examples of groupings
grouped = df.groupby('place_cleaned')['place'].unique()
grouped_multiple = grouped[grouped.apply(len) > 1]

print("Exemples de lieux regroupés après nettoyage :")
for cleaned, originals in grouped_multiple.head(10).items():
    print(f"Nettoyé: '{cleaned}'")
    print(f"Originaux: {list(originals)}")
    print("---")

df = df.drop(columns=['place'])

# Enregistrer le parquet avec la colonne 'place' nettoyée
df.to_parquet('data/STEP04_earthquakes.parquet')

In [ ]:
df = pd.read_parquet('data/STEP04_earthquakes.parquet')

# Nombre de valeurs uniques dans 'place' original
unique_original = df['place_cleaned'].nunique()
print(f"Nombre de valeurs uniques dans 'place' original : {unique_original}")


### Ajout colonne magnitude uniformisé

In [ ]:

def convert_magnitude(row):
    mag = row["mag"]
    mtype = row["magType"]

    if pd.isna(mag) or pd.isna(mtype):
        return np.nan

    mtype = str(mtype).lower()  # normalisation

    if mtype in ["ml", "mlg", "mlr"]:  
        return mag

    if mtype in ["md"]:
        return 0.85 * mag + 0.3

    if mtype in ["mw", "mwc", "mwb", "mwr", "mww"]:
        return (mag - 0.9) / 0.67

    if mtype in ["mc"]:
        return 0.85 * mag + 0.3

    if mtype in ["ma"]:
        return mag  

    return pd.NA

    
import numpy as np

df = pd.read_parquet('data/STEP04_earthquakes.parquet')

# Appliquer la fonction de conversion
df['mag_uniform'] = df.apply(convert_magnitude, axis=1)

# Sauvegarder dans un nouveau parquet
df.to_parquet('data/STEP05_earthquakes.parquet')

print("Nouveau fichier 'data/STEP05_earthquakes.parquet' créé avec la colonne 'mag_uniform'.")

### Suppression colonnes

In [ ]:
df = pd.read_parquet('data/STEP05_earthquakes.parquet')

df = df.drop(columns=['net', 'locationSource', 'magSource', 'status', 'dmin'])

df.to_parquet('data/STEP06_earthquakes.parquet')

### Réorganisation du parquet

In [ ]:
# Charger ton fichier
df = pd.read_parquet("data/STEP06_earthquakes.parquet")

# Dictionnaire de renommage
rename_map = {
    "id": "ID",
    "time": "date",
    "updated": "date_maj_infos",

    "depth": "profondeur_km",

    "mag": "magnitude",
    "magType": "type_magnitude",
    "magError": "erreur_magnitude",
    "magNst": "nb_stations_magnitude",

    "nst": "nb_stations_localisation",
    "gap": "ecart_azimut",
    "rms": "rms",

    "horizontalError": "erreur_horiz",
    "depthError": "erreur_profondeur",

    "place_cleaned": "lieu",
    "mag_uniform": "mag_uniforme"
}

# Renommer les colonnes
df = df.rename(columns=rename_map)

# Nouvel ordre des colonnes
new_order = [
    "ID", "date","lieu","magnitude", "type_magnitude","latitude", "longitude", "profondeur_km",  "mag_uniforme","nb_stations_localisation", "nb_stations_magnitude","ecart_azimut", "rms","erreur_horiz","erreur_profondeur","erreur_magnitude","reseau","type", "date_maj_infos"
]

# Garder uniquement les colonnes existantes
new_order = [col for col in new_order if col in df.columns]

# Réorganiser
df = df[new_order]

df.to_parquet('data/STEP07_earthquakes.parquet')


### Suppression des évènements autres que "earthquake" et suppression de la colonne "type"

In [4]:
df = pd.read_parquet('data/STEP07_earthquakes.parquet')

# Filtrer pour garder uniquement les événements de type 'earthquake'
df = df[df['type'] == 'earthquake']

# Supprimer la colonne 'type'
df = df.drop(columns=['type'])

# Sauvegarder dans un nouveau parquet
df.to_parquet('data/STEP08_earthquakes.parquet')

print("Fichier 'data/STEP08_earthquakes.parquet' créé avec uniquement les événements 'earthquake' et colonne 'type' supprimée.")

Fichier 'data/STEP08_earthquakes.parquet' créé avec uniquement les événements 'earthquake' et colonne 'type' supprimée.


### Garder de 2000 à 2005 uniquement

In [6]:
# Filtrer les lignes dont la date est comprise entre 2000 et 2005 inclus
df = pd.read_parquet('data/STEP08_earthquakes.parquet')

mask_2000_2005 = (df['date'] >= '2000-01-01') & (df['date'] < '2006-01-01')
df_2000_2005 = df[mask_2000_2005]

print(f"Nombre de lignes entre 2000 et 2005 : {len(df_2000_2005)}")
df_2000_2005.head()

df_2000_2005.to_parquet('data/STEP09_earthquakes.parquet')

Nombre de lignes entre 2000 et 2005 : 541525


### Garder uniquement les USA

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

df = pd.read_parquet("data/STEP09_earthquakes.parquet")

# Convertir en GeoDataFrame (WGS84)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

# Charger les pays
world = gpd.read_file(
    "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
)

usa = world[world["ADMIN"] == "United States of America"]

# Projection métrique US
gdf_m = gdf.to_crs("EPSG:5070")   # mètres
usa_m = usa.to_crs("EPSG:5070")

# Buffer de 50 km autour des USA
usa_buffer = usa_m.geometry.buffer(50 * 1000)

# Filtrage spatial
mask = gdf_m.geometry.within(usa_buffer.iloc[0])
filtered = gdf.loc[mask].copy()

# Export parquet
filtered.drop(columns="geometry").to_parquet("data/STEP10_earthquakes.parquet")

print(f"✅ {len(filtered)} lignes exportées")


✅ 380123 lignes exportées


### Ressenti ?

In [22]:
df = pd.read_parquet('data/STEP10_earthquakes.parquet')

# Création de la colonne "ressenti" selon les critères donnés
def determine_ressenti(mag_unif, prof):
    # Si magnitude uniformisée et profondeur sont tous les deux NaN, retourner NaN
    if pd.isna(mag_unif) and pd.isna(prof):
        return pd.NA
    
    # Si magnitude uniformisée > 4, c'est ressenti peu importe la profondeur
    if pd.notna(mag_unif) and mag_unif > 4.0:
        return 'oui'
    
    # Si magnitude uniformisée > 3 ET profondeur < 20, c'est ressenti
    if pd.notna(mag_unif) and pd.notna(prof) and mag_unif > 3.0 and prof < 20:
        return 'oui'
    
    # Sinon non ressenti
    return 'non'

df['ressenti'] = df.apply(lambda row: determine_ressenti(row['mag_uniforme'], row['profondeur_km']), axis=1)

# Placer la colonne "ressenti" en avant-dernière position
cols = list(df.columns)
cols.insert(-1, cols.pop(cols.index('ressenti')))
df = df[cols]

df.to_parquet('data/STEP11_earthquakes.parquet')
print("Colonne 'ressenti' créée et fichier 'data/STEP11_earthquakes.parquet' sauvegardé.")

Colonne 'ressenti' créée et fichier 'data/STEP11_earthquakes.parquet' sauvegardé.


### Création d'un petit parquet

In [23]:
df = pd.read_parquet('data/STEP11_earthquakes.parquet')

# Extraire les 10 premières lignes
df_lite = df.head(10)

# Sauvegarder en parquet lite
df_lite.to_parquet('data/earthquakes_lite.parquet')

print("Fichier 'data/earthquakes_lite.parquet' créé avec les 10 premières lignes.")

Fichier 'data/earthquakes_lite.parquet' créé avec les 10 premières lignes.


### Valeurs manquantes

In [19]:
df = pd.read_parquet('data/STEP11_earthquakes.parquet')

# Compter les valeurs manquantes par colonne
missing_values = df.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(missing_values)

Nombre de valeurs manquantes par colonne :
ID                               0
date                             0
lieu                             0
magnitude                    20342
type_magnitude               20405
latitude                         0
longitude                        0
profondeur_km                    0
mag_uniforme                 61435
nb_stations_localisation     97532
nb_stations_magnitude       177082
ecart_azimut                 90784
rms                           2589
erreur_horiz                141364
erreur_profondeur             5037
erreur_magnitude            221474
ressenti                    375876
date_maj_infos                   0
dtype: int64
